# LangChain

## Setup

In [1]:
%pip install langchain langchain_openai langchain_community langchainhub langgraph

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/23/98/c70fac0f1b3193ced86013b563119c27c68ac26b684815f407555224108d/langchain-0.1.0-py3-none-any.whl.metadata
  Using cached langchain-0.1.0-py3-none-any.whl.metadata (13 kB)
  Obtaining dependency information for langchain_openai from https://files.pythonhosted.org/packages/f4/ba/6cfe8bdcc0c4f6b420d9f6128d1970273764eb1a7c2e3ef02efa2271dd30/langchain_openai-0.0.2-py3-none-any.whl.metadata
  Using cached langchain_openai-0.0.2-py3-none-any.whl.metadata (570 bytes)
  Obtaining dependency information for langchain_community from https://files.pythonhosted.org/packages/3b/db/ce4a9244a6a2d9fc07394f94fe4c7041a478cc14e03072f9e8bbece92667/langchain_community-0.0.10-py3-none-any.whl.metadata
  Using cached langchain_community-0.0.10-py3-none-any.whl.metadata (7.3 kB)
  Obtaining dependency information for langchainhub from htt


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import os

# load api keys from openai_api_key.txt and tavily_api_key.txt files
with open(os.path.expanduser("../api_key.txt")) as f:
    openai_api_key = f.read().strip()

with open(os.path.expanduser("../tavily_api_key.txt")) as f:
    tavily_api_key = f.read().strip()

# set api keys
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["TAVILY_API_KEY"] = tavily_api_key

In [2]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.callbacks.tracers import ConsoleCallbackHandler

## LangChain Example

In [11]:
# Select the tools that will be used by the agent
tools = [TavilySearchResults(max_results=3)]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

# Construct the OpenAI Functions agent
agent = create_openai_functions_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [9]:
question = "What is the weather in Rome?"

In [12]:
agent_executor.invoke({"input": f"{question}"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in Rome'}`


[{'url': 'https://weatherspark.com/h/y/71779/2024/Historical-Weather-during-2024-in-Rome-Italy', 'content': '2024 Weather History in Rome Italy  Cloud Cover in 2024 in Rome Observed Weather in 2024 in Rome Hours of Daylight and Twilight in 2024 in Rome  Moon Rise, Set & Phases in 2024 in Rome Humidity Comfort Levels in 2024 in Rome Wind Speed in 2024 in Rome  Rome Temperature History 2024 Hourly Temperature in 2024 in Rome Compare Rome to another city:The daily range of reported temperatures (gray bars) and 24-hour highs (red ticks) and lows (blue ticks), placed over the daily average high (faint red line) and low (faint blue line) temperature, with 25th to 75th and 10th to 90th percentile bands. Hourly Temperature in 2024 in Rome Link Download'}, {'url': 'https://www.weather25.com/europe/italy/lazio/rome', 'content': 'will give you an idea of weather trends in Rome. Fo

{'input': 'What is the weather in Rome?',
 'output': 'I found some resources with information about the weather in Rome. You can check the weather forecast for Rome on the following websites:\n1. [WeatherSpark](https://weatherspark.com/h/y/71779/2024/Historical-Weather-during-2024-in-Rome-Italy): Provides historical weather data for Rome, including cloud cover, observed weather, hours of daylight, moon phases, humidity, and wind speed.\n2. [Weather25](https://www.weather25.com/europe/italy/lazio/rome): Offers weather trends for Rome by month and provides a 14-day weather forecast.\n3. [Met Office](https://www.metoffice.gov.uk/weather/forecast/sr2y7gxtj): Provides a seven-day weather forecast for Rome.\n\nYou can visit these websites for more detailed information about the weather in Rome.'}

## Custom Prompts

In [3]:
simple_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "let's think step by step to get the CORRECT answer"),
])

simple_prompt

ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="let's think step by step to get the CORRECT answer"))])

## ScientificGPT

In [5]:
sys_neuro = "You are an expert researcher in Neuroscience. You provide insights on the research in the Neuroscience field. You are a member of the Neuroscience community. Your only knowledge is Neuroscience. Ignore any facts that are not related to Neuroscience. Your only job is to give your opinion using just your knowledge of Neuroscience. Reflect on the question from your own perspective. You are not an expert in all fields, just Neuroscience."

sys_math = "You are an expert researcher in Mathematics. You provide insights on the research in the Mathematics field. You are a member of the Mathematics community. Ignore any facts that are not related to Mathematics. Your only job is to give your opinion using just your knowledge of Mathematics. Do not use any other knowledge that you may have. Reflect on the question from your own perspective. You are not an expert in all fields, just Mathematics."

sys_drafter = "You are a research supervisor. Your job is to supervise the research work of your multi-disciplinary team. You will be asked to provide connections between different fields of research. You will be provided with the thoughts of your colleagues and you will put together their ideas. Write a technical report that summarizes your team's answers."

sys_ML = "You are an expert researcher in Machine Learning. You are a member of the Machine Learning community. Your only knowledge is Machine Learning. Ignore any facts that are not related to Machine Learning. You will be given a research question and a summary of your colleagues' thoughts. Your job is to suggest a novel research method that is inspired by the ideas of your colleagues. You are not an expert in all fields, just Machine Learning. After elaborating on the method, provide a set of experiments that can be used to test the method. You don't neet to code the experiments."

sys_engineer = "You are an engineer. Your job is to code the experiments provided by your colleagues. Use the Python programming language and the PyTorch library."

In [ ]:
research_question = "How could we build a new very-long-term memory mechanism for a neural network? What would be the best way to do it? Provide a detailed answer."

### Implementation

In [6]:
res_neuro_prompt = ChatPromptTemplate.from_messages([
    ("system", f"{sys_neuro}"),
    ("human", "{input}"),
])

res_math_prompt = ChatPromptTemplate.from_messages([
    ("system", f"{sys_math}"),
    ("human", "{input}"),
])

res_drafter_prompt = ChatPromptTemplate.from_messages([
    ("system", f"{sys_drafter}"),
    ("human", "Initial Question: {input}\n\nNeuroscience expert: {NS_response}\n\nMath expert: {MT_response}"),
])

res_ML_prompt = ChatPromptTemplate.from_messages([
    ("system", f"{sys_ML}"),
    ("human", "Research Question: {input}\n\nSummary of colleagues' thoughts: {report}"),
])

res_engineer_prompt = ChatPromptTemplate.from_messages([
    ("system", f"{sys_engineer}"),
    ("human", "{ML_response}"),
])

model = ChatOpenAI(model="gpt-4")

In [7]:
neuro_chain = res_neuro_prompt | model
math_chain = res_math_prompt | model

drafter_chain = {
    "input": lambda x: x["input"],
    "NS_response": neuro_chain | StrOutputParser(),
    "MT_response": math_chain | StrOutputParser()
} | res_drafter_prompt | model

ml_chain = {
    "input": lambda x: x['input'],
    "report": drafter_chain | StrOutputParser(),
} | res_ML_prompt | model

engineer_chain = {
    "ML_response": ml_chain | StrOutputParser()
} | res_engineer_prompt | model

### Demo

In [9]:
print(engineer_chain.invoke({"input": research_question}, config={'callbacks': [ConsoleCallbackHandler()]}))

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "How could we build a new very-long-term memory mechanism for a neural network? What would be the best way to do it? Provide a detailed answer."
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<ML_response>] Entering Chain run with input:
{
  "input": "How could we build a new very-long-term memory mechanism for a neural network? What would be the best way to do it? Provide a detailed answer."
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<ML_response> > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "How could we build a new very-long-term memory mechanism for a neural network? What would be the best way to do it? Provide a detailed answer."
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<ML_response> > 3:chain:RunnableSequence > 4:chain:RunnableParallel<input,report>] Entering Chain run with input:
{
  "input": "How

## Need Cycles? Use Graphs.

_Source: https://github.com/langchain-ai/langgraph_

In [17]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

In [31]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.agents import AgentFinish

agent = RunnablePassthrough.assign(
    agent_outcome = agent_runnable
)

# Define the function to execute tools
def execute_tools(data):
    # Get the most recent agent_outcome - this is the key added in the `agent` above
    agent_action = data.pop('agent_outcome')

    # Get the tool to use
    tool_to_use = {t.name: t for t in tools}[agent_action.tool]
    # Call that tool on the input
    observation = tool_to_use.invoke(agent_action.tool_input)
    # We now add in the action and the observation to the `intermediate_steps` list
    # This is the list of all previous actions taken and their output
    if('intermediate_steps' not in data):
        data['intermediate_steps'] = list()
    data['intermediate_steps'].append((agent_action, observation))
    return data

# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
    # If the agent outcome is an AgentFinish, then we return `exit` string
    # This will be used when setting up the graph to define the flow
    if isinstance(data['agent_outcome'], AgentFinish):
        return "exit"
    # Otherwise, an AgentAction is returned
    # Here we return `continue` string
    # This will be used when setting up the graph to define the flow
    else:
        return "continue"

In [32]:
from langgraph.graph import END, Graph

workflow = Graph()

# Add the agent node, we give it name `agent` which we will use later
workflow.add_node("agent", agent)
# Add the tools node, we give it name `tools` which we will use later
workflow.add_node("tools", execute_tools)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        # If `tools`, then we call the tool node.
        "continue": "tools",
        # Otherwise we finish.
        "exit": END
    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('tools', 'agent')

# Finally, we compile it! 
# This compiles it into a LangChain Runnable, 
# meaning you can use it as you would any other runnable
chain = workflow.compile()

In [42]:
question = "What is the current weather in Rome?"

In [43]:
execution = chain.invoke({"input": f"{question}", "intermediate_steps": []})

In [44]:
print(f"input: {execution['input']}\n\n")
for i, step in enumerate(execution['intermediate_steps']):
    print(f"{i+1}. Tool: {step[0].tool}")
    print(f"tool_input: {step[0].tool_input}")
    print(f"tool_output: {step[1]}\n")
print(f"\noutput: {execution['agent_outcome'].log}")

input: What is the current weather in Rome?


1. Tool: tavily_search_results_json
tool_input: {'query': 'current weather in Rome'}
tool_output: [{'url': 'https://hikersbay.com/weather/italy/rome?lang=en', 'content': '1008 73% 3.12 55% Rome Weather forecast for Tuesday, January 9, 2024 Rome weather Tomorrow at 1:00 AM overcast clouds  12℃ 53℉ 10℃ 49℉ Rome Weather forecast for Monday, January 8, 2024 Rome weather Today at 7:00 PM few clouds 12℃ 53℉  1021 68% 2.17 100% Rome Weather forecast for Wednesday, January 10, 2024  1018 73% 3.3 6% Rome Weather forecast for Saturday, January 13, 2024 Rome weather Saturday at 1:00 AM sky is clearCurrent weather and long-term 45 days forecast ☃ Weather phenomena recorded in previous years in Rome ☀ Detailed weather forecast for the next 10 days ☔ Long term weather forecast for Rome for 30 days ... Rome Weather forecast for Thursday, January 4, 2024 The day lasts 9 hours 13 minutes Sunrise: 7:39 AM Sunset: 4:52 PM ... Rome Weather ...'}]


output: The